In [ ]:
#hide
%load_ext autoreload
%autoreload 2

In [ ]:
# default_exp figures

# ds18b20.figures

> Create figures using plotly to be used with dash.

In [ ]:
#hide
from nbdev.showdoc import *

In [ ]:
#export
import plotly.graph_objects as go
import numpy as np
from scipy.interpolate import interp1d
import pandas as pd
from pathlib2 import Path
import datetime

## Fill line between markers - interp1d

In [ ]:
#export
def new_x_y(x, y, n, kind='linear'):
    """
    Returns an interpolation of type kind f with n data points based on y = f(x)
    
    x, y: arrays defining y = f(x)
    n: the number of interpolated date points returned
    kind: type of interpolation ('linear', 'nearest', 'zero', 'slinear', 'quadratic', 'cubic',
    'previous', 'next', where 'zero', 'slinear', 'quadratic' and 'cubic')
    """
    f_x = interp1d(x, y)
    x_new = np.linspace(x[0], x[-1], n)
    y_new = f_x(x_new)
    return x_new, y_new

In [ ]:
#export
def add_trace(fig, x, y, sz, cs, cmin, cmax, hi=None, sl=True):
    fig.add_trace(
        go.Scatter(
            x=x,
            y=y,
            mode='markers',
            marker={
                'color': y,
                'cmin': cmin,
                'cmax': cmax,
            'colorscale': cs, 
            'size': sz,
        },
        hoverinfo=hi,
        showlegend=sl,
        )
    )

In [ ]:
#export
def get_figure(x, y, n, sz1, sz2, cs, cmin, cmax, convert_ts=False):
    x_new, y_new = new_x_y(x, y, n)
    if convert_ts:
        x = pd.to_datetime(x, unit='s')
        x_new = pd.to_datetime(x_new, unit='s')
    fig = go.Figure()
    add_trace(fig, x_new, y_new, sz1, cs, cmin, cmax, hi='skip', sl=False)
    add_trace(fig, x, y, sz2, cs, cmin, cmax, sl=False)
    return fig

In [ ]:
# define a color scale
colorscale=[
    [0, "rgb(49,54,149)"],
    [0.1111111111111111, "rgb(69,117,180)"],
    [0.2222222222222222, "rgb(116,173,209)"],
    [0.3333333333333333, "rgb(171,217,233)"],
    [0.4444444444444444, "rgb(224,243,248)"],
    [0.5555555555555556, "rgb(254,224,144)"],
    [0.6666666666666666, "rgb(253,174,97)"],
    [0.7777777777777778, "rgb(244,109,67)"],
    [0.8888888888888888, "rgb(215,48,39)"],
    [1.0, "rgb(165,0,38)"]
]

In [ ]:
t = np.linspace(0, 10, 50)
x, y = t, (np.cos(t) * 0.5 + 0.5) * 30

fig = get_figure(x, y, 1000, 5, 8, colorscale, 10, 35)

fig.update_layout(plot_bgcolor='white', margin=dict(t=0, b=40, r=0, l=0, pad=0),
              xaxis=dict(showgrid=False, zeroline=False, mirror=True, linecolor='gray'),
              yaxis=dict(showgrid=False, zeroline=False, mirror=True, linecolor='gray'),
                hovermode="x")

fig

## Example - Real data figure

In [ ]:
temp_file = Path('webapp3/data/get_temp_C.out')
df = pd.read_csv(temp_file, sep=' ', header=None, names=['dev_sn', 'date', 'time', 'temp_raw', 'temp_C'], parse_dates=[['date', 'time']])
df.tail()

,date_time,dev_sn,temp_raw,temp_C
51696,2021-02-24 10:53:03.808198,28-032197791b3c,18.687,20.403
51697,2021-02-24 10:54:03.209276,28-03219779d339,18.937,19.878
51698,2021-02-24 10:54:04.097442,28-032197791b3c,18.687,20.403
51699,2021-02-24 10:55:02.489345,28-03219779d339,18.937,19.878
51700,2021-02-24 10:55:03.367491,28-032197791b3c,18.750,20.467


In [ ]:
df2 = df[df.dev_sn == '28-03219779d339'].tail(720).iloc[::-20]
df2['ts'] = (df2.date_time.values.astype(np.int64) // 10 ** 9).tolist()

x = df2.ts.values
y = df2.temp_C.values

fig = get_figure(x, y, 2000, 3, 3, colorscale, 19, 21, convert_ts=True)

fig.update_layout(
    xaxis_title="Datum",
    yaxis_title="Temperatur / °C",
    template='none',
    #autosize=False,
    #width=1000,
    #height=150,
    margin = dict(
        l = 100,
        r = 10,
        b = 100,
        t = 20,
        pad = 0,
    ),
    hovermode="x"
)

fig.show()